# КМ-01 | Романецький Микита | Лаб 4

In [ ]:
import random
import time

import pika

### Завдання 1: 
1. Змініть код <span style="color:#1E90FF">Producer'а</span>, щоб він генерував інший текст повідомлень за Вашим 
вибором. 
2. Змініть назву черги повідомлень як у коді <span style="color:#1E90FF">Producer'а</span>, так і в коді <span style="color:#FFD500">Cоnsumer'а</span>  
та переконайтеся, що вони використовують однакову назву черги. 
3. Змініть код <span style="color:#1E90FF">Producer'а</span>, щоб генерувати повідомлення з іншим інтервалом. 
4. Змініть код <span style="color:#FFD500">Cоnsumer'а</span>, щоб роздрукувати кількість повідомлень, які він 
отримав, на додаток до вмісту повідомлення. 

### Producer (Виробник)

In [ ]:
def task_1_producer():
    counter = 1
    QUEUE = 'Romanetskiy'

    while True:
        time_to_sleep = random.randint(500, 5000) / 1000  # 0.5 - 5 секунд 
        time.sleep(time_to_sleep)

        connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
        channel = connection.channel()

        channel.queue_declare(queue=QUEUE, durable=False, exclusive=False, auto_delete=False)

        message = f'Завдання 1 | Повідомлення {counter}'
        body = message.encode('utf-8')

        channel.basic_publish(exchange='', routing_key=QUEUE, body=body)

        print(f'[LOG] Повідомлення N {counter} | (TTS = {time_to_sleep})')
        counter += 1


try:
    task_1_producer()
except KeyboardInterrupt:
    print('[LOG] Зупинено')

### Consumer (Споживач)

<details>
  <summary>Натисніть, щоб розгорнути</summary>

  _Код закоментований, тому що на захисті буду запускати Producer та Consumer паралельно_

</details>

In [ ]:
# def task_1_consumer():
#     global msg_counter
#     msg_counter = 1
#     QUEUE = 'Romanetskiy'

#     connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
#     channel = connection.channel()

#     def callback(ch, method, properties, body):
#         global msg_counter
#         print(f'Отримано повідомлення N {msg_counter}: {body.decode("utf-8")}')
#         msg_counter += 1

#     channel.queue_declare(queue=QUEUE, durable=False, exclusive=False, auto_delete=False)

#     channel.basic_consume(queue=QUEUE, on_message_callback=callback, auto_ack=True)

#     print('\n[LOG] Початок відстеження черги')
#     channel.start_consuming()


# try:
#     task_1_consumer()
# except KeyboardInterrupt:
#     print('[LOG] Зупинено')

<br/>

### Завдання 2 | Варіант 16 : 

##### Створіть прямий обмін із трьома прив’язаними до нього чергами  
1-а черга має отримувати повідомлення з ключем маршрутизації <span style="color:red"> «alert:emergency» </span>  
2-а черга має отримувати повідомлення з ключем маршрутизації <span style="color:#FF4500"> «alert:urgent» </span>  
3-я черга має отримувати повідомлення з ключем маршрутизації <span style="color:orange"> «alert:warning» </span>  

Додайте нове прив’язування до обмінника, щоб усі повідомлення з ключем  
маршрутизації <span style="color:gold"> «alert» </span> також надсилалися до черги <span style="color:orange"> «alert:warning» </span>


### Producer (Виробник)

In [ ]:
def task_2_producer():
    counter = 1
    exchange_name = 'direct_alerts'

    connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
    channel = connection.channel()

    # Створення прямого обміну
    channel.exchange_declare(exchange=exchange_name, exchange_type='direct')

    while True:
        time_to_sleep = 3
        time.sleep(time_to_sleep)

        # Генерування повідомлення з відповідним ключем маршрутизації
        message = f"   \t Завдання 2 | Повідомлення {counter}"
        routing_key = random.choice(['alert:emergency', 'alert:urgent', 'alert:warning', 'alert'])
        body = message.encode('utf-8')

        # Надсилання повідомлення до прямого обміну з вказаним ключем маршрутизації
        channel.basic_publish(exchange=exchange_name, routing_key=routing_key, body=body)

        print(f'[LOG] Надіслано повідомлення {counter} | {routing_key}')
        counter += 1


try:
    task_2_producer()
except KeyboardInterrupt:
    print('[LOG] Зупинено')

### Consumer (Споживач)

<details>
  <summary>Натисніть, щоб розгорнути</summary>

  _Код закоментований, тому що на захисті буду запускати Producer та Consumer паралельно_

</details>

In [ ]:
# def task_2_consumer():
#     connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
#     channel = connection.channel()

#     exchange_name = 'direct_alerts'

#     channel.exchange_declare(exchange=exchange_name, exchange_type='direct')

#     def emergency_callback(ch, method, properties, body):
#         print(f"Отримано повідомлення з ключем маршрутизації 'alert:emergency': {body.decode('utf-8')}")

#     def urgent_callback(ch, method, properties, body):
#         print(f"Отримано повідомлення з ключем маршрутизації 'alert:urgent': {body.decode('utf-8')}")

#     def warning_callback(ch, method, properties, body):
#         print(f"Отримано повідомлення з ключем маршрутизації 'alert:warning': {body.decode('utf-8')}")

#     # Черга для 'alert:emergency'
#     emergency_queue = 'emergency_queue'
#     channel.queue_declare(queue=emergency_queue, durable=False)
#     channel.queue_bind(exchange=exchange_name, queue=emergency_queue, routing_key='alert:emergency')
#     channel.basic_consume(queue=emergency_queue, on_message_callback=emergency_callback, auto_ack=True)

#     # Черга для 'alert:urgent'
#     urgent_queue = 'urgent_queue'
#     channel.queue_declare(queue=urgent_queue, durable=False)
#     channel.queue_bind(exchange=exchange_name, queue=urgent_queue, routing_key='alert:urgent')
#     channel.basic_consume(queue=urgent_queue, on_message_callback=urgent_callback, auto_ack=True)

#     # Черга для 'alert:warning'
#     warning_queue = 'warning_queue'
#     channel.queue_declare(queue=warning_queue, durable=False)
#     channel.queue_bind(exchange=exchange_name, queue=warning_queue, routing_key='alert:warning')
#     channel.queue_bind(exchange=exchange_name, queue=warning_queue, routing_key='alert')
#     channel.basic_consume(queue=warning_queue, on_message_callback=warning_callback, auto_ack=True)

#     print('\n[LOG] Початок відстеження черг')
#     channel.start_consuming()


# try:
#     task_2_consumer()
# except KeyboardInterrupt:
#     print('[LOG] Зупинено')

<br/>

### Завдання 3 | Варіант 16 :  

##### Створіть обмінник для веб-сайту електронної комерції  
Ключі маршрутизації мають формат `"product.<category>.<action>"`  
Категорією може бути <span style="color:#1E90FF"> електроніка, одяг, книги, тощо</span>  
Дією може бути- <span style="color:#FFD500"> create, update чи delete</span>    

Майте єдину чергу, яка підписується на обмінник, щоб відстежувати всі зміни (update) продукту.

### Producer (Виробник)

In [ ]:
def task_3_producer():
    counter = 1
    connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
    channel = connection.channel()

    channel.exchange_declare(exchange='product_exchange', exchange_type='topic')

    while True:
        time_to_sleep = random.randint(1, 3)
        time.sleep(time_to_sleep)

        category = random.choice(['electronics', 'clothing', 'books'])  # список можливих категорій
        action = random.choice(['create', 'update', 'delete'])  #  список можливих дій

        routing_key = f"product.{category}.{action}"
        message = f'Продукт {action} в категорії {category}'
        body = message.encode('utf-8')

        channel.basic_publish(exchange='product_exchange', routing_key=routing_key, body=body)

        print(f'[LOG] Надіслано повідомлення {counter}\t| {routing_key}   \t| {message}')
        counter += 1


try:
    task_3_producer()
except KeyboardInterrupt:
    print('[LOG] Зупинено')

### Consumer (Споживач)

<details>
  <summary>Натисніть, щоб розгорнути</summary>

  _Код закоментований, тому що на захисті буду запускати Producer та Consumer паралельно_

</details>

In [ ]:
# def task_3_consumer():
#     connection = pika.BlockingConnection(pika.ConnectionParameters('localhost'))
#     channel = connection.channel()

#     def callback(ch, method, properties, body):
#         print(f'Отримано повідомлення: {body.decode("utf-8")}')

#     channel.exchange_declare(exchange='product_exchange', exchange_type='topic')
#     channel.queue_declare(queue='product_changes')

#     # Видалення всіх існуючих зв'язків для queue
#     for rk in ['product.*.*', 'product.*.update']:
#         channel.queue_unbind(queue='product_changes', exchange='product_exchange', routing_key=rk)

#     channel.queue_bind(exchange='product_exchange', queue='product_changes', routing_key='product.*.*')
#     # channel.queue_bind(exchange='product_exchange', queue='product_changes', routing_key='product.*.update')

#     channel.basic_consume(queue='product_changes', on_message_callback=callback, auto_ack=True)

#     print('\n[LOG] Початок відстеження черги')
#     channel.start_consuming()


# try:
#     task_3_consumer()
# except KeyboardInterrupt:
#     print('[LOG] Зупинено')

<br/>

### Висновки  

У ході лабораторної роботи ознайомився із базовими принципами роботи брокера повідомлень RabbitMQ та його  використання для асинхронного обміну повідомленнями. В процесі були вивчені основні концепції такі, як Producer,  Consumer, обмінники та черги.  

Завдання дозволили провести практичні вправи із зміною коду, створенням прив’язаних черг, та роботою із ключами  маршрутизації. Результатом стала практична робота із реальними сценаріями використання брокера повідомлень для  організації асинхронного обміну повідомленнями в розподілених системах.
